In [1]:
from utils_art import *

from sklearn.datasets import make_classification
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import Isomap
from sklearn.neighbors import KNeighborsTransformer
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_regression

import pandas as pd
import numpy as np
from textblob import TextBlob
import tempfile
# np.random.seed(0)

from IPython.core.display import Image as image
from PIL import Image
from IPython.display import Image
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.cm as cm

TAG_SELECTION_LIST = ["NNP","NN","JJ"]

import text_analysis
import warnings
import re
warnings.filterwarnings('ignore')
ts = text_analysis.text_analysis()


imported 'utils_art'



Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Embedding Functions

In [2]:
def dfColumnToMatrix(df,data_col="o_data",max_index=1000,index_col="hash_key",matToDf=True) :
    max_index = min(max_index,df.shape[0])
    mat_data = df[data_col].apply(np.array).to_numpy()[0:max_index]
    mat_index = []
    if index_col != "" :
        mat_index = df[index_col].apply(np.array).to_numpy()[0:max_index]
    mat_out = []
    for i in range(max_index) :
        split_list = mat_data[i].strip('[]').split(',')
        if index_col != "" :
            mat_out.append([mat_index[i]] + split_list)
        else :
            mat_out.append([i] + split_list)
    if matToDf :
        mat_out_df = pd.DataFrame(mat_out)
        mat_out_df.set_index(0, inplace=True)
        if index_col != "" :
            mat_out_df.index.rename(index_col, inplace=True)
        else :
            mat_out_df.index.rename('index', inplace=True)
        return mat_out_df
    else :
        return mat_out
    
def extractEmbedding(df,number_entries=1000): #51372
    df_out = dfColumnToMatrix(df,data_col="o_data",max_index=number_entries,index_col="hash_key",matToDf=True) #
    return df_out

### Keyword Functions

In [11]:
def parse_keywords_list(df, column_name="keywords_list",titleFlag=False,entry_limit=1000,output_df=False):#51400
    out = df[column_name].to_numpy()[0:entry_limit]
    union_list = [[] for i in range(entry_limit)]
    count = 0
    for entry in out :
        if not titleFlag :
            parsed_list = decomposeKeywordList(entry)#.replace("'","").lower().strip("][").split(', ') 
        else :
            parsed_list = decomposeTitle(entry)
        union_list[count] = parsed_list
        count = count +1
    if output_df :
        return union_list #
    else :
        return union_list

def flattenMatrix(mat):
    out_list = []
    for li in mat :
        out_list = out_list + li
    return out_list
def getMostCommunKeywords(union_list_np, source_limit=100,return_word_list=True,display_stats=True) :
    df = pd.DataFrame(union_list_np, columns=['keyword'])
    df = df['keyword'].value_counts().to_frame("count").sort_values(by=['count'],ascending=True)
    if display_stats :
        display(df)
    count_sum_before = df["count"].sum()
    entry_sum_before = df.shape[0]
    source_limiy_count = int(df.iloc[[int(-min(entry_sum_before,source_limit))]]["count"].tolist()[0])
    df = df[df['count'].between(source_limiy_count, 100000000)]
    count_sum_after = df["count"].sum()
    entry_sum_after = df.shape[0]
    if display :
        print("Unique keywords :",entry_sum_before,"  Sum of occurences :",count_sum_before,"  ("+str(round(count_sum_before/entry_sum_before,2))+" avg)")
        print("Unique keywords :",entry_sum_after,"  Sum of occurences :",count_sum_after,"  ("+str(round(count_sum_after/entry_sum_after,2))+" avg)")
    if return_word_list :
        df2 = df.index.to_numpy()
        return df2
    else :
        return df

def cleanString(string) :
    #~out_str = string.strip("][”“|’><%—–//").replace("'", "").replace("\\d", "");
    out_str = "".join(re.findall("[a-zA-Z]+", string))
    if len(out_str)>3:
        return out_str
    else :
        return ""

def decomposeTitle(string, getList=True) :
    decomposed = "=".join(re.findall("[a-zA-Z]+", string.lower()))
    if getList :
        decomposed = decomposed.split('=')
    return decomposed

def decomposeKeywordList(string,getList=True) :
    decomposed = "&".join(re.findall("[a-zA-Z]+", string.lower())) #.split(', ')
    if getList :
        decomposed = decomposed.split('&')
    return decomposed
#     blob = TextBlob(str(string))
#     word_list = []
#     for wordT in blob.tags :
#         if wordT[1] in TAG_SELECTION_LIST :
#             string2 = str(cleanString(wordT[0].lower()))
#             if string2 != "" :
#                 word_list.append(string2)
#     return word_list

def df_setup(par_list) :
    df = pd.DataFrame(par_list)#
    col_count = df.shape[1]
    df.rename(columns=lambda x: str(x), inplace=True)#
    df['word_count'] = df.apply(lambda x: x.count(), axis=1)#
    df = df.fillna(value="")#df = df.fillna(value="#")
    df["0"] = df["0"].astype(str)
    df['word_combined'] = df[[str(i) for i in range(col_count)]].agg(' '.join, axis=1)
    #df['word_combined'].replace(" #","", inplace=True)
    display(df)
    print(df[["word_count"]].describe())
    return df



def genrateKeywordExtract(df_main, entry_limit=10, common_word_max = 10, column_name="keywords_list", isTitle = False):
    par_list = parse_keywords_list(df_main,column_name,isTitle,entry_limit)
    df = df_setup(par_list)
    par_list = flattenMatrix(par_list)
    out = getMostCommunKeywords(par_list,common_word_max)
    print(out)
    out = set(out)
    row_list = []
    for i in range(entry_limit) :
        row_list.append(list(out & set(df.loc[i, :].values.flatten().tolist())))
    df_min = df_setup(row_list)
    df = df.join(df_min, how="inner", lsuffix='_f', rsuffix='_s')
    print(df.dtypes)
    #df["word_combined_f"].replace(" @","")
    #df["word_combined_s"].replace(" @","")
    #df['word_combined_s'] = df['word_combined_s'].str.replace(' @','')
    #df['word_combined_f'] = df['word_combined_s'].str.replace(' @','')
    df['word_combined_f'] = df['word_combined_f'].str.replace(r'\s+', ' ', regex=True)
    df['word_combined_s'] = df['word_combined_s'].str.replace(r'\s+', ' ', regex=True)
    return df

def generateNLPonKeywords(df_main,index_from=0,index_to=500,display_log=True):
    if index_to == -1 :
        index_to = df_main.shape[0]
    df_np = df_main["word_combined_all"].apply(np.array).to_numpy()[0:index_to]
    df_hash = df_main["hash_key"].apply(np.array).to_numpy()[0:index_to]
    mat_index = []
    for i in range(index_from,index_to) :
        if display_log :
            print(" - Generate NLP for article #"+str(i)+"  (char:"+str(len(df_np[i]))+")")
        nlp_dict = ts.analyseText2(str(df_np[i]),True)
        mat_index.append(nlp_dict|{"hash_key":df_hash[i]})
    df = pd.DataFrame(mat_index, columns = list(mat_index[0].keys())) 
    df.set_index("hash_key", inplace=True)
    df_main.set_index("hash_key", inplace=True)
    display(df)
    display(df_main)
    df = df_main.join(df,on='hash_key', how="inner",lsuffix='_k')
    return df

### Category Functions

### Variables

In [12]:
folder_path_input = "C:/Users/User/OneDrive/Desktop/article/files_3/1_4_join_main/arc/"
filename_input = "join_main_big"
folder_path_embd_df="C:/Users/User/OneDrive/Desktop/article/files_3/2_1_embdedding_main/embd_df/"
folder_path_embd_join="C:/Users/User/OneDrive/Desktop/article/files_3/2_1_embdedding_main/embd_join/"
folder_path_embd_raw="C:/Users/User/OneDrive/Desktop/article/files_3/2_1_embdedding_main/test/"
filename_embd_df="embd_out_main_testfinal"
filename_embd_join="embd_all_main_testfinal"
filename_embd_raw_save="embedding_matrix_max"
filename_embd_raw_load="embedding_matrix_XXXX"

folder_path_keyword_out="C:/Users/User/OneDrive/Desktop/article/files_3/2_2_keyword_main/out/"
folder_path_keyword_out_nlp="C:/Users/User/OneDrive/Desktop/article/files_3/2_2_keyword_main/out_nlp/"
filename_keyword_out="keywords_df_main_test"
filename_keyword_out_nlp="keywords_df_nlp_main_test"

# # folder_path="C:/Users/User/OneDrive/Desktop/article/file_2/test_llm_output/run2/"
# folder_path_embd="C:/Users/User/OneDrive/Desktop/article/file_2/join_2_df/embd/only_embd/"
# folder_path_keyword="C:/Users/User/OneDrive/Desktop/article/file_2/join_2_df/keyword/"
# folder_path_llm="C:/Users/User/OneDrive/Desktop/article/file_2/join_2_df/llm/"
# folder_path_graph="C:/Users/User/OneDrive/Desktop/article/file_2/join_2_df/graphs/"
# folder_path_viz="C:/Users/User/OneDrive/Desktop/article/file_2/join_2_df/viz/"
# \files_3\1_4_join_main\arc
# filename="article_stats_embedding"
# filename_embd="embedding_matrix_1000"#"embedding_matrix_main"#"article_stats_embedding_smain" #""
# filename_keyw="article_keyword_main"
# open_path_keyw="C:/Users/User/OneDrive/Desktop/article/file_2/join_1_df/"

## Main Embedding Workflow

In [5]:
df_main = openDFcsv(folder_path_embd_df,filename_embd_df)
emb_mat = extractEmbedding(df_main,51373)
saveDFcsv(emb_mat, folder_path_embd_raw, filename_embd_raw_save,True)

True

## Main Keyword Workflow

In [13]:
df_main = openDFcsv(folder_path_input,filename_input)
entry_limit=10000# 51372
common_word_max = 500

df1 = genrateKeywordExtract(df_main,entry_limit,common_word_max,"title_quer",True)
df2 = genrateKeywordExtract(df_main,entry_limit,common_word_max,"keywords_list",False)

df = df1.join(df2, how="inner", lsuffix='_t', rsuffix='_k')
#df = df[["0_f_t","1_f_t","3_f_t","word_count_f_t","word_combined_f_t","0_f_k","1_f_k","3_f_k","word_count_f_k","word_combined_f_k","0_s_t","1_s_t","3_s_t","word_count_s_t","word_combined_s_t","0_s_k","1_s_k","3_s_k","word_count_s_k","word_combined_s_k"]]
df = df[["word_count_f_t","word_combined_f_t","word_count_f_k","word_combined_f_k","word_count_s_t","word_combined_s_t","word_count_s_k","word_combined_s_k"]]#,"0_s_k","0_s_t"]]
df['word_combined_all'] = df[["word_combined_f_t","word_combined_f_k"]].agg(' '.join, axis=1)
df['word_count_all'] = df["word_count_f_t"]+df["word_count_f_k"]

display(df)
print(df.dtypes)
df = df_main.join(df, how="inner")
saveDFcsv(df, folder_path_keyword_out_nlp, filename_keyword_out_nlp,True)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,word_count,word_combined
0,top,enlisted,airman,addresses,key,cannon,issues,during,visit,air,...,,,,,,,,,12,top enlisted airman addresses key cannon issue...
1,top,chef,secrets,revealed,from,the,judges,mouths,to,your,...,,,,,,,,,14,top chef secrets revealed from the judges mout...
2,top,of,the,pops,the,new,yorker,,,,...,,,,,,,,,7,top of the pops the new yorker ...
3,why,top,predators,matter,an,in,depth,look,at,new,...,,,,,,,,,13,why top predators matter an in depth look at n...
4,white,wizzard,over,the,top,review,angry,metal,guy,,...,,,,,,,,,9,white wizzard over the top review angry metal ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,antipatterns,that,are,derailing,technology,transformations,mckinsey,,,,...,,,,,,,,,7,antipatterns that are derailing technology tra...
9996,five,books,about,technology,and,government,mit,technology,review,,...,,,,,,,,,9,five books about technology and government mit...
9997,expert,tips,for,successful,construction,technology,implementation,autodesk,redshift,,...,,,,,,,,,9,expert tips for successful construction techno...
9998,termite,fishing,chimpanzees,provide,clues,to,the,evolution,of,technology,...,,,,,,,,,15,termite fishing chimpanzees provide clues to t...


        word_count
count  10000.00000
mean      11.27650
std        3.53201
min        3.00000
25%        9.00000
50%       11.00000
75%       13.00000
max       27.00000


,count
keyword,
geti,1
pistol,1
bronkhorst,1
coelho,1
lao,1
...,...
news,1636
in,1773
s,1979


Unique keywords : 14915   Sum of occurences : 112765   (7.56 avg)
Unique keywords : 507   Sum of occurences : 67876   (133.88 avg)
['lab' 'trump' 'role' 'car' 'under' 'opinion' 'brings' 'programs' 'j'
 'rise' 'internet' 'rights' 'trade' 'great' 'finale' 'changed' 'military'
 'men' 'nuclear' 'soccer' 'chronicle' 'services' 'training' 'rankings'
 'bureau' 'little' 'cleveland' 'design' 'again' 'dean' 'gallup' 'decade'
 'columbia' 'savannah' 'variety' 'fund' 'steve' 'mlb' 'phoenix' 'display'
 'her' 'slate' 'think' 'newshour' 'system' 'relations' 'movies' 'billion'
 'vanderbilt' 'ranks' 'monitor' 'baseball' 'morning' 'know' 'white' 'hit'
 'tax' 'cities' 'james' 'through' 'class' 'film' 'prospects' 'helps'
 'hall' 'learning' 'release' 'pbs' 'goes' 'young' 'secret' 'california'
 'ocregister' 'stats' 'x' 'airforce' 'ranking' 'stanford' 'finds'
 'colorado' 'earth' 'oklahoman' 'north' 'tonight' 'official' 'rumors'
 'jazeera' 'they' 'rock' 'michael' 'changing' 'yorker' 'project'
 'government' 'ta

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,word_count,word_combined
0,link,during,top,air,force,,,,,,,,,,,,,,5,link during top air force
1,your,to,from,chef,top,weekly,the,entertainment,news,,,,,,,,,,9,your to from chef top weekly the entertainment...
2,of,yorker,top,new,the,,,,,,,,,,,,,,5,of yorker top new the
3,an,research,in,com,look,top,why,new,at,,,,,,,,,,9,an research in com look top why new at
4,white,top,over,the,review,,,,,,,,,,,,,,5,white top over the review
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,are,that,mckinsey,technology,,,,,,,,,,,,,,,4,are that mckinsey technology
9996,about,and,government,technology,mit,five,review,,,,,,,,,,,,7,about and government technology mit five revie...
9997,for,technology,,,,,,,,,,,,,,,,,2,for technology
9998,of,to,technology,university,the,news,,,,,,,,,,,,,6,of to technology university the news


...

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,word_count,word_combined
0,leaders,air,force,working,,,,,,,,,,,,4,leaders air force working
1,shes,judges,chef,times,season,,,,,,,,,,,5,shes judges chef times season
2,life,art,york,work,,,,,,,,,,,,4,life art york work
3,worlds,species,look,research,,,,,,,,,,,,4,worlds species look research
4,point,white,good,old,track,review,,,,,,,,,,6,point white good old track review
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,tools,value,business,major,organizations,technology,bank,,,,,,,,,7,tools value business major organizations techn...
9996,books,innovation,nations,technology,national,science,state,,,,,,,,,7,books innovation nations technology national s...
9997,team,teams,technology,success,,,,,,,,,,,,4,team teams technology success
9998,tools,skills,study,technology,,,,,,,,,,,,4,tools skills study technology


         word_count
count  10000.000000
mean       5.672300
std        2.260757
min        0.000000
25%        4.000000
50%        6.000000
75%        7.000000
max       15.000000
0_f                object
1_f                object
2_f                object
3_f                object
4_f                object
5_f                object
6_f                object
7_f                object
8_f                object
9_f                object
10_f               object
11_f               object
12_f               object
13_f               object
14_f               object
15                 object
16                 object
17                 object
18                 object
19                 object
20                 object
21                 object
word_count_f        int64
word_combined_f    object
0_s                object
1_s                object
2_s                object
3_s                object
4_s                object
5_s                object
6_s                object
7_s           

,word_count_f_t,word_combined_f_t,word_count_f_k,word_combined_f_k,word_count_s_t,word_combined_s_t,word_count_s_k,word_combined_s_k,0_s_k,0_s_t,word_combined_all,word_count_all
0,12,top enlisted airman addresses key cannon issue...,15,roy cannon key airman working enlisted chief a...,5,link during top air force,4,leaders air force working,leaders,link,top enlisted airman addresses key cannon issue...,27
1,14,top chef secrets revealed from the judges mout...,14,mouths padma eric revealed gail times shes che...,9,your to from chef top weekly the entertainment...,5,shes judges chef times season,shes,your,top chef secrets revealed from the judges mout...,28
2,7,top of the pops the new yorker,11,art warhol york gallery factory life soup work...,5,of yorker top new the,4,life art york work,life,of,top of the pops the new yorker art warhol yor...,18
3,13,why top predators matter an in depth look at n...,14,matter predators look species endangered popul...,9,an research in com look top why new at,4,worlds species look research,worlds,an,why top predators matter an in depth look at n...,27
4,9,white wizzard over the top review angry metal ...,11,old white review good track wizzard iron bands...,5,white top over the review,6,point white good old track review,point,white,white wizzard over the top review angry metal ...,20
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,7,antipatterns that are derailing technology tra...,13,technology tools delivery derailing cost platf...,4,are that mckinsey technology,7,tools value business major organizations techn...,tools,are,antipatterns that are derailing technology tra...,20
9996,9,five books about technology and government mit...,11,technology nations intellectual national scien...,7,about and government technology mit five review,7,books innovation nations technology national s...,books,about,five books about technology and government mit...,20
9997,9,expert tips for successful construction techno...,12,technology teams team success successful exper...,2,for technology,4,team teams technology success,team,for,expert tips for successful construction techno...,21
9998,15,termite fishing chimpanzees provide clues to t...,15,technology termitefishing chimpanzee termites ...,6,of to technology university the news,4,tools skills study technology,tools,of,termite fishing chimpanzees provide clues to t...,30


word_count_f_t        int64
word_combined_f_t    object
word_count_f_k        int64
word_combined_f_k    object
word_count_s_t        int64
word_combined_s_t    object
word_count_s_k        int64
word_combined_s_k    object
0_s_k                object
0_s_t                object
word_combined_all    object
word_count_all        int64
dtype: object


True

In [ ]:
min_index = 0
max_index = 200 
df_main = openDFcsv(folder_path_keyword_out,filename_keyword_out)
df = generateNLPonKeywords(df_main,min_index,max_index)
saveDFcsv(df,folder_path_keyword_out_nlp,filename_keyword_out_nlp) # , mode="w"
display(df)
print(df.dtypes)

In [ ]:
### Load Joinned DataFrame
df_main = openDFcsv(folder_path,filename)

# 1 gen # 2 save
        # df = extractEmbedding()
        # saveDFcsv(df, folder_path_embd, filename_embd,True)

# 3 load
df_embd = openDFcsv(folder_path_embd, filename_embd)
df_embd.drop(["hash_key"], axis=1,inplace=True)


# 4 sellect
df_main = df_main.loc[(df_main['category'] == "WORLD")| (df_main['category'] == "NATION") | (df_main['category'] == "TECHNOLOGY")]#(df_main['category'] == "TOP") |( 

# 4 dim red
df_red = dimReduction_TSNE(df_embd, n_components=2, perplexity=30, early_exaggeration =4.0,learning_rate=1000,n_iter=1000,random_state=0)
        # df_red = dimReduction_PCA(df, n_components=2, svd_solver="auto",tol=0.0,random_state=0)
        # df_red = dimReduction_IPCA(df, n_components=2, batch_size=100)
        
# 5 join with main
df_main = df_main.join(df_red, how="inner")
display(df_main)
print(df_main.dtypes)
plot2D(df_main,createdefconfdict()|_2d_embd)
# 5 chose column (1 to 5)
# 7 cal centroids or GMM


In [ ]:
def decomposeSentenceList(stringList) :
    word_list_out = []
    tag_list_out = []
    for sent in stringList :
        blob = TextBlob(str(sent))
        word_list = []
        tag_list = []
        #print(blob.tags)
        for wordT in blob.tags :
            string2 = str(cleanString(wordT[0])).lower()
            if string2 != "" :
                word_list_out.append(string2)
                tag_list_out.append(wordT[1])
        # word_list_out.append(word_list)
        # tag_list_out.append(tag_list)
#             if wordT[1] in TAG_SELECTION_LIST :
#                 string2 = str(cleanString(wordT[0].lower()))
#                 if string2 != "" :
#                     word_list.append(string2)
    return word_list_out,tag_list_out

x,y= decomposeSentenceList(stringList[:])
z = getMostCommunKeywords(y,50)
print(list(z))
for word in x :
    print(TextBlob(str(word)).tags[0])

In [ ]:
stringList = ["""An MIT Technology Review Series: AI Colonialism - MIT Technology Review""",
"""Before Nobels: Gifts to and from rich patrons were early science's currency - The Conversation""",
"""Establishment of a COVID-19 mRNA vaccine technology transfer hub to scale up global manufacturing - World Health Organization (WHO)""",
"""Small Business Spotlight: Simcoach Games | PA Department of Community & Economic Development - PA Department of Community & Economic Development""",
"""The Big Bang Theory boss teases changes ahead in season 10 - Entertainment Weekly News""",
"""Social Security, joblessness top economic concerns for Americans - PBS NewsHour""",
"""$5 million gift names Miami Herbert Business School's graduate career advancement center - University of Miami: News@theU""",
"""How to Organize After Occupy Wall Street - The Nation""",
"""Google explains top traits of its best teams - Business Insider""",
"""The Blissful and Trivial Life: How Entertainment Deprives a Soul - Desiring God""",
"""Analysis: Should Judges Exit Fundraising Business? - The Texas Tribune""",
"""The Flash brings back Rick Cosnett, Matt Letscher, Jessica Parker Kennedy - Entertainment Weekly News""",
"""Hispanic and Latino students get an inside look at science at Argonne - Argonne National Laboratory""",
"""Is It Wrong to Mourn Paris More Deeply Than Beirut? - The Nation""",
"""Robert Folger Named Endowed Scholar in Business Ethics - UCF""",
"""Stony Brook University--SUNY - Profile, Rankings and Data | US News Best Colleges - U.S. News & World Report""",
"""Think entertainment is violent today? The Victorians were much, much worse - The Conversation""",
"""Top Chef All-Stars Episode 10: Imagine Pronouncing "Target" in a French Accent - Eater""",
"""Most Epic Wins and Losses in Sports, '12 - Business Insider""",
"""Google CEO Sundar Pichai Responds to Job Application From 7-Year-Old - Business Insider""",
"""Xavier Nation Magazine Feature: Game Plan - Xavier University Athletics""",
"""Best GMs in Sports - Bleacher Report""",
"""The #1 Office Perk? Natural Light - HBR.org Daily""",
"""Business Gets Bigger Even as Americans Prefer Small - Gallup.com""",
"""Update: 22 Clinton emails deemed â€˜top secretâ€™ by government - PBS NewsHour""",
"""Top 130 Sports Facility Architecture Firms for 2023 - Building Design + Construction""",
"""Black Mirror's Hated in the Nation creates a world where everyone is at fault - The Verge""",
"""Lieberman: Secret Service scandal "just gets more troubling" - CBS News""",
"""'X-Men: Apocalypse': Sophie Turner says how her Jean Grey will be different - Entertainment Weekly News""",
"""The World's Muslims: Religion, Politics and Society - Pew Research Center""",
"""Moving inventions to the marketplace - MIT News""",
"""IU study finds brain differences in athletes playing contact vs. noncontact sports - IU Newsroom""",
"""Mountain DewÂ®. DoritosÂ®. THOUSANDS of Xbox One Entertainment Systems. "Every 2 Minutes." - PR Newswire""",
"""U.S lag in science, math a disaster in the making - CNN""",
"""Animal Logic, Syco Entertainment Partner - Computer Graphics World""",
"""SunTrust Names Ashleigh Overly As Sports and Entertainment Managing Director - musicrow.com""",
"""Team Rubicon recruits medical volunteers to serve Navajo Nation - VA News - news.va.gov""",
"""Keenan Reynolds is Tennessean Sports Person of Year - Tennessean""",
"""No PhDs needed: how citizen science is transforming research - Nature.com""",
"""Elon ranks among country's top 40 undergraduate business programs - Today at Elon""",
"""The Amazonian tribe defending their land with technology - Al Jazeera English""",
"""Turning GenAI Magic into Business Impact | BCG - BCG""",
"""Women in Science at Grand Canyon (U.S - National Park Service""",
"""Reimagining growth: a new approach to business success - The Guardian""",
"""Discovery and technology for human health - Nature.com""",
"""Red Sox 2020 Top 30 Prospects list - MLB.com""",
"""LSB Economic Department Faculty Promotions - News - Leavey School of Business - SCU - Santa Clara University""",
"""Garret Dillahunt reacts to John Dorie shocker on 'Fear The Walking Dead' - Entertainment Weekly News""",
"""Students' Summer Experiences - News - Leavey School of Business - SCU - Santa Clara University""",
"""Commentary on 2.0 GPA policy: Sports part of a good education - Rockford Register Star""",
"""EPA tells nation's dirty power plants to clean up - The Times""",
"""SDIS21: What are the challenges in making tech more sustainable? - World Economic Forum""",
"""The Wachovia Whistleblower - The Nation""",
"""World's Oldest Leather Shoe Foundâ€”Stunningly Preserved - National Geographic""",
"""David Miliband: How to Bring Peace to the World's Longest Civil War - TIME""",
"""Scientists to track twisters in world's largest tornado study | NCAR & UCAR News - news.ucar.edu""",
"""Bipedal robot developed at Oregon State achieves Guinness World Record in 100 meters - Oregon State University""",
"""CIA Unveils Lab to Focus on In-House Technology Innovation - FedTech Magazine""",
"""Top budget officials examine encroachment around Cherry Point - DVIDS""",
"""Cervical cancer - World Health Organization (WHO)""",
"""Balaji Rajagopalan named dean of Robert J. Trulaske, Sr. College of Business // Show Me Mizzou - University of Missouri""",
"""Dominican Republic Overview: Development news, research, data - World Bank Group""",
"""Different Disneylands around the world - BBC.com""",
"""Kevin Feige talks Phase 5, Kang, and the future of the MCU - Entertainment Weekly News""",
"""Daedalic Entertainment Ending Internal Development, Work on New Lord of the Rings Game Halted - IGN""",
"""Syrian refugees settling in to nation's schools - The Texas Tribune""",
"""Teenage Conflict and Digital Media - Pew Research Center""",
"""Ryan Seacrestâ€™s Civic Entertainment Group Acquires Marketing Agency Culture Shop - Hollywood Reporter""",
"""HBS Appoints Terrill L. Drake Chief Diversity and Inclusion Officer - hbs.edu""",
"""World Cup fans didnâ€™t trigger Zika outbreak in Brazil, study says - PBS NewsHour""",
"""South China Sea: International Contest Over the World's Most Important Waterway | TIME - TIME""",
"""Chinatown Will Recover From the Coronavirus - The Nation""",
"""Paxton No Stranger to Controversial Business Deals - The Texas Tribune""",
"""The Masters: Men's Golf Earns Top Ranking as Season Begins - Vanderbilt University News""",
"""â€˜We Have Not a Governmentâ€™: The US Before the Constitution - The Nation""",
"""EXPLAINER: This year's four new Olympic sports, broken down - The Associated Press""",
"""Can science predict criminal behavior? | Penn Today - Penn Today""",
"""Enhancing mung bean hydration using the ultrasound technology: description of mechanisms and impact on its ... - Nature.com""",
"""Forgot the name of the game top-down fighting game for PS1 - GameDev.net""",
"""Press Release Announcing U.S. Recognition of Israel (1948) - National Archives |""",
"""Why do school sports start before classes do? - CNN""",
"""Entertainment briefs - Bedford - DATM News""",
"""Choctaw Nation Small Business Development receives Excellence in Entrepreneurship Award - Choctaw Nation""",
"""Inside Edward Jonesâ€™ Take on Technology - ThinkAdvisor""",
"""Liberia: Getting Back to the Founding Faith - ChristianityToday.com""",
"""Air Force's top NCO gets look at all things Edwards AFB - Air Force Link""",
"""Science News Correspondent Ann Gibbons Wins 2012 Anthropology in Media Award - AAAS""",
"""World's hottest temperature cools a bit | ASU News - ASU News Now""",
"""Top 10 Highly Concentrated Industries - IBISWorld - Cision News""",
"""Who are the oldest athletes in sports after Brady retires? - The Providence Journal""",
"""2018â€™s Top Tech Trends - American Libraries""",
"""World Patient Safety Day 2023 - World Health Organization (WHO)""",
"""Senate Confirms Berhe as Federal Office of Science Director | Newsroom - news.ucmerced.edu""",
"""Supporting Israel's â€œstart-up nationâ€ - The Keyword | Google Product and Technology News""",
"""NFL - Spending an NFL Sunday with a sports betting syndicate - ESPN""",
"""Top 5 things to know about wild horse and burro fertility control - Bureau of Land Management""",
"""Verizon Business Introduces Cloud-based 'Storage-as-a-Service' - Verizon""",
"""Can Technology Make You Happy? - IEEE Spectrum""",
"""Entertainment One Picks-Up Leslie Bibb Comedy 'Take Care' - TheWrap""",
"""THR's Power Business Managers 2013 â€“ The Hollywood Reporter - Hollywood Reporter""",
"""SOUTHWEST CAPTURES SENIOR LEAGUE BASEBALL WORLD SERIES, DEFEATS CENTRAL 8-1 - littleleague.org""",
"""Business, community leader Seth Atwood dead at 92 - Rockford Register Star""",
"""New national survey shows 70% of Americans are sports fans - Spectrum News""",
"""Why Some 'Top Chef' Viewers Want the Show to Expand Its Palate - Food & Wine""",
"""Patriots Scandal Continues Long Debate Over Sports Ethics - The New York Times""",
"""Actresses, Business Leaders and Other Wealthy Parents Charged in U.S. College Entry Fraud (Published 2019) - The New York Times""",
"""Caesars Entertainment Provides Update on Plan to Reopen Properties - PR Newswire""",
"""Ryan Reynolds on why Deadpool worked and Green Lantern failed - Entertainment Weekly News""",
"""How the world embraced consumerism - BBC.com""",
"""Top Cheffer Brian Hill Gets Into the LA Food Truck Game - Eater""",
"""Central Banking and Fintechâ€”A Brave New World? - International Monetary Fund""",
"""Advice from the Top > United States Marine Corps Flagship > News Display - Marines.mil""",
"""Game Designer Stands at Rowdy Intersection of Entertainment and Art - WIRED""",
"""Farrakhan's Nation of Islam to Argue UFOs Are Real - Fox News""",
"""Nissan licenses autonomous driving technology to Hitachi Construction Machinery Company - global.nissannews.com""",
"""Why the World Will End (Again) on Friday - Livescience.com""",
"""CHART OF THE DAY: In Case You Had Any Doubts About Where Google's Revenue Comes From - Business Insider""",
"""Chapter 4: Evolution and Perceptions of Scientific Consensus - Pew Research Center""",
"""The intersection of law and sports - Sports Business Journal""",
"""For Sports Employees, Work-Life Balance Is A Challenge - Texas A&M University Today""",
"""Twenty-seven UW faculty listed among 'world's most influential scientific minds' by Thomson Reuters - washington.edu""",
"""Steve Jobs Death: Tech and Entertainment Industry Reacts - Hollywood Reporter""",
"""Teen Drinking Down In State, Nation - WPR""",
"""Sent into the World: Jesus' Mission and Ours - Desiring God""",
"""U.S. News & World Report Ranks Embry-Riddle Worldwide One of Nation's Best Online Educators - ERAU News""",
"""AQ Top 5 Latin American Art Activists: Edel Rodriguez - Americas Quarterly""",
"""As Jasper Highlands grows, Top of the Rock restaurant expands overlook dining, adds pizzeria - Chattanooga Times Free Press""",
"""The Future of Sports Journalism - UNLV NewsCenter""",
"""Pacers Sports & Entertainment Celebrates Jim Morris on World Food Programme Nobel Peace Prize - NBA.com""",
"""Olympics, Inc: Inside The Secretive, $6 Billion World Of The International Olympic Committee - Business Insider""",
"""Conference realignment poses risks to big-time college sports - Inside Higher Ed""",
"""President Obama honors exemplary math and science teachers - National Science Foundation""",
"""Graduate Costume Technology - UNCSA - UNCSA""",
"""Some countries are shrinking the gender pay gap - World Economic Forum""",
"""Jane Fonda to be Honored at THRâ€™s Women in Entertainment Breakfast - Hollywood Reporter""",
"""Walt Disney World Resort to Host Professional and Amateur Sports This Spring - The Walt Disney Company""",
"""Public safety relies on behind-the-scenes technology - Route Fifty""",
"""Mercy Named a Top Stroke Center in Nation - Mercy""",
"""A Reign Not of This World - The Nation""",
"""Entertainment Buzz: Facebook's top 10 moments of the week - CBS News""",
"""A Primer On Snapchat And Its Demographics - Business Insider""",
"""Nation's top muscular dystrophy leaders join patients to share ideas - UT Southwestern""",
"""Jameson Crane Sports Medicine Institute Celebrates Five Years |Ohio State Medical Center - Wexner Medical Center - The Ohio State University""",
"""Promoting Multilateral Solutions for a Globalized World - International Monetary Fund""",
"""Ware College House to become 'hub for technology' - The Daily Pennsylvanian""",
"""The business class where high school students are running the company - PBS NewsHour""",
"""Food as entertainment at Hash House a Go Go - San Diego Reader""",
"""Where science and nonsense collide - Nature.com""",
"""'Jurassic World' director on where the franchise can go next - Entertainment Weekly News""",
"""NFL Draft Sparks Conversations Around NCAA Sports Betting Rule - The Oberlin Review""",
"""Study Finds Athletes Who Play Indoor Sports at Risk of Vitamin D Deficiency - George Mason University""",
"""Six Technologies and Innovations for Women's Health - agora""",
"""World Obesity Day 2022 â€“ Accelerating action to stop obesity - World Health Organization (WHO)""",
"""The Shutdown Corner Interview: Jimmy Johnson - Yahoo Sports""",
"""The Top 15 Emerging Artists of 2015 - Artsy""",
"""Casey Patterson, Entertainment Industry Powerhouse and Cocreator of Lip Sync Battle, Takes Us Inside Her Tribeca ... - Architectural Digest""",
"""Sargent Graduate Programs Rated in Top 25 | BU Today - Boston University""",
"""High value awards turn spotlight back on science - Nature.com""",
"""Peter Parker preps for action in exclusive 'Spider-Man: Into the Spider-Verse' image - Entertainment Weekly News""",
"""A few more thoughts about Marquee Sports Network's Cubs game broadcasts - Bleed Cubbie Blue""",
"""The technology imperative for life sciences companies - McKinsey""",
"""Bruce Campbell says things 'go bad real quick' in 'Ash vs Evil Dead' season 3 - Entertainment Weekly News""",
"""Eastern Michigan University Clinical Research program again ranks among nation's best - EMU Today""",
"""'The Twist' tops 'Billboard's All-Time Top 100 songs - USA TODAY""",
"""Chicago Cabbies Fight for a Fairer Fare - The Nation""",
"""Postdoc Named L'OrÃ©al USA For Women in Science Fellow - Caltech""",
"""Best Career Advice: Insights from Successful Top Achievers - SUCCESS Magazine""",
"""Lucid's World-Leading Electric Powertrain Technology Propels Aston Martin to a Bold Electric Future | Lucid Group, Inc. - Lucid Group""",
"""Industry: Open for business - Nature.com""",
"""Science Operations Configuration - NASA Mars - NASA Mars Exploration""",
"""Science education: Research on the reservation - Nature.com""",
"""Review: Glitchy Two Worlds II Is a Massive World of Nothing - WIRED""",
"""How Big-Time Sports Ate College Life - The New York Times""",
"""Sony Computer Entertainment Launches PlayStationÂ® Business In People's Republic Of China With Release Of ... - PR Newswire""",
"""Computer Science for Non-Majors Takes Many Forms - The New York Times""",
"""Investors Are Betting Big on Carbon Removal Technology. The Reality Is More Complicated. - TIME""",
"""Penn State Health one of six health systems in nation to add physical activity as a vital sign - Penn State Health News""",
"""How Technology is Changing the Way Athletes Train - Smithsonian Magazine""",
"""World Report 2022: Ukraine - Human Rights Watch""",
"""â€œIs, ought, shouldâ€â€”scientists' role in discourse on the ethical and social implications of science and technology ... - Nature.com""",
"""ACC Selects Elevate Sports Ventures To Advise On Revenue, Ticketing, and Fan Experience Strategy - The ACC""",
"""How to Fix College Sports - Center For American Progress""",
"""Fracking Our Food Supply - The Nation""",
"""Launch of New Citizen Science project, Active Asteroids! - Science@NASA""",
"""How Painting Skills Landed John Douglas On Stage Drumming For ZZ Top And Aerosmith - Houston Public Media""",
"""Introducing The Atlantic's New Science Section - The Atlantic""",
"""How To Empower Retail Employees With Technology - Shopify""",
"""iPic EntertainmentÂ® Expands in the Sunshine State - PR Newswire""",
"""Doering: Top five Indianapolis sports venues - The Butler Collegian""",
"""Cedar Point adding sports park complex - Detroit Free Press""",
"""Thor: Ragnarok: Meet Cate Blanchett's Hela, the franchise's first female villain - Entertainment Weekly News""",
"""GE CEO Jeff Immelt on becoming top 10 software company - Business Insider""",
"""To Grow Your Business Abroad, Partner with Local Influencers - HBR.org Daily""",
"""How deadly was the poison gas of WW1? - BBC.com""",
"""Complex issue: Protect women's sports while welcoming trans athletes - USA TODAY""",
"""Schools' Merger To Bring Arts, Humanities, Technology Together - The University of Texas at Dallas""",
"""The Mood of the World - Gallup""",
"""The Top 100 Convenience Store Chains of 2018 - CSNews Online""",
"""Plasma Thruster Technology Developed at GW Licensed by Vector | GW Today | The George Washington University - GW Today""",
"""Lebanese Armed Forces Remain Top-Notch Security Partner - Department of Defense""",
"""NFL Rights Help The Yahoo Sports App Score Millions Of Users - AdExchanger""",
"""Advising and Support - Health and Exercise Science - chhs.colostate.edu""",
"""Diablo Cody didn't 'quit' Madonna biopic, final script turned in - Entertainment Weekly News""",
"""Opinion | A World Without Coral Reefs - The New York Times""",
"""Body mass index and breast cancer: analysis of a nation-wide population-based prospective cohort study on 1 393 985 ... - Nature.com""",
"""Technology of the Year: Apple's iPad - ZDNet""",
"""How preferential is world trade? - CEPR""",
"""Nation's third-highest honor bestowed upon brave Marine - DVIDS""",
"""â€˜Mall Cops: Mall of Americaâ€™ Shopping for entertainment? New show filmed here falls short - St. Paul Pioneer Press""",
"""The Bizarre Disdain for High School Sports - EducationNext"""]



In [14]:
from textblob.wordnet import Synset
octopus = Synset("octopus.n.02")
shrimp = Synset("shrimp.n.03")
octopus.path_similarity(shrimp)

0.1111111111111111

In [15]:
from nltk.corpus import wordnet
from nltk.corpus import wordnet as wn
print(wn.synsets('dog')[0])
print(wn.synsets('dog', pos=wn.VERB))
print(wn.synset('dog.n.01').definition())
print(wn.synset('dog.n.01').examples())
# print(wn.synset('dog.n.01').lemmas())
# print(wn.synset('dog.n.01').lemma_names('ita'))


Synset('dog.n.01')
[Synset('chase.v.01')]
a member of the genus Canis (probably descended from the common wolf) that has been domesticated by man since prehistoric times; occurs in many breeds
['the dog barked all night']


In [16]:
dog = wn.synset('live.n.01')
cat = wn.synset('die.n.01')
dog.path_similarity(cat)

WordNetError: No lemma 'live' with part of speech 'n'

In [17]:
print(wn.synsets('dog')[-1])

Synset('chase.v.01')


In [18]:
print(wn.synsets('fuck'))

[Synset('fuck.n.01'), Synset('sleep_together.v.01')]


In [19]:
def similarity(st1,st2,met=0):
    word1 = wn.synsets(st1)[0]
    word2 = wn.synsets(st2)[0]
    print(word1,word2)
    sim1 = word1.path_similarity(word2)
    sim2 = word1.lch_similarity(word2)
    sim3 = word1.wup_similarity(word2)
    return sim1,sim2,sim3
#     if met==0 :
#         return sim1

In [20]:
print(similarity("science","technology"))
print(similarity("games","community"))
print(similarity("miami","verge"))
print(similarity("director","fear"))

Synset('science.n.01') Synset('technology.n.01')
(0.08333333333333333, 1.1526795099383855, 0.35294117647058826)
Synset('game.n.01') Synset('community.n.01')
(0.1, 1.3350010667323402, 0.3076923076923077)
Synset('miami.n.01') Synset('brink.n.01')
(0.07142857142857142, 0.9985288301111273, 0.13333333333333333)
Synset('director.n.01') Synset('fear.n.01')
(0.07142857142857142, 0.9985288301111273, 0.13333333333333333)
